In [27]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import pickle
from scipy.cluster import hierarchy 
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import fcluster

plt.rcParams['lines.linewidth']=2.5
plt.style.use("fivethirtyeight")
plt.rcParams["figure.max_open_warning"] = 99
np.set_printoptions(precision=2, edgeitems=3, linewidth=1000000)

In [3]:
signal_numbers = list(range(0, 500))

In [4]:
with open('dist_mat_condensed_SORTED.pkl', 'rb') as f:
    dist_mat = pickle.load(f)

In [33]:
def plot_mapping(xys, sNums):
    plt.style.use("fivethirtyeight")
    plt.figure(figsize=(8,4))
    plt.xticks(fontsize=8)
    plt.yticks(fontsize=8)
    
    for sNum in sNums:
        x,_,y,_ = xys[sNum]
        plt.plot(x, y, lw='1.5', label=sNum)
    
    plt.xlabel("FreqVector", fontsize=8)
    plt.ylabel("Velocity FFT Amplitude", fontsize=8)

    plt.subplots_adjust(bottom=0.20)        
    plt.legend(loc='upper right', fontsize=8)
    plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%0.2f'))

### Dendrogram

In [26]:
%matplotlib widget
linkage_matrix = linkage(dist_mat, "single")
plt.figure(figsize=(9,4))
plt.xlabel('Signal Number')
plt.xticks(fontsize=2)
dendrogram(linkage_matrix, labels = signal_numbers, leaf_font_size=9)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
max_d = 145 # the cutoff point (threshold for clustering)

In [8]:
clusters = fcluster(linkage_matrix, max_d, criterion='distance')  # use cut-off distance, max_d
print(f"# of clusters: {len(np.unique(clusters))}")

# of clusters: 3


In [9]:
df_clusters = pd.DataFrame(columns=['signal_num', 'cluster'])
df_clusters['signal_num'] = list(range(500))
df_clusters['cluster'] = clusters

In [10]:
# number of signals in each cluster
df_clusters.cluster.value_counts()

1    493
2      6
3      1
Name: cluster, dtype: int64

In [11]:
# signals in cluster 3
df_clusters[df_clusters.cluster==3]

,signal_num,cluster
98,98,3


In [12]:
# signals in cluster 2
df_clusters[df_clusters.cluster==2]

,signal_num,cluster
48,48,2
49,49,2
99,99,2
100,100,2
101,101,2
102,102,2


### Timestamps

In [18]:
with open('xyst.pkl', 'rb') as f:
    xyst = pickle.load(f)

In [19]:
# check timestamps of speciic cluster
xyst.loc[df_clusters.cluster==3][['t']]

,t
98,2020-05-20 08:42:16.475


In [20]:
xyst.loc[df_clusters.cluster==2][['t']]

,t
48,2020-05-19 05:12:14.965
49,2020-05-19 05:42:14.934
99,2020-05-20 09:12:17.005
100,2020-05-20 09:42:17.940
101,2020-05-20 10:12:17.727
102,2020-05-20 10:42:17.604


In [21]:
# or check timestamp of particular signals
xyst.loc[[98, 99, 100, 101, 102]][['t']]

,t
98,2020-05-20 08:42:16.475
99,2020-05-20 09:12:17.005
100,2020-05-20 09:42:17.940
101,2020-05-20 10:12:17.727
102,2020-05-20 10:42:17.604


### 3D Plot

In [23]:
%matplotlib widget

In [22]:
xys=[]
for r in range(0, len(xyst)):
    tx = [float(i) for i in xyst.iloc[r]['x'][1:-1].split(', ')]
    ta = [float(i) for i in xyst.iloc[r]['a'][1:-1].split(', ')]
    tv = [float(i) for i in xyst.iloc[r]['v'][1:-1].split(', ')]
    tt = xyst.iloc[r]['t']
    xys.append([tx, ta, tv, tt])
print('extracted')

extracted


In [25]:
fig = plt.figure(figsize=(9,5))
ax = fig.gca(projection='3d')

num_of_signals = len(xys)

# use "range_num_of_signals = range(num_of_signals)" to plat all signals 
range_num_of_signals =  [95, 96, 97, 98, 99, 100, 101, 102] 

len_f = len(xys[0][0])

for ix in range_num_of_signals:
    f,_,v,t = xys[ix]
    y=np.ones(len_f) * (ix)
    ax.plot3D(f, y, v, lw=1)

ytls = ["#"+str(ytl)+" @ "+xyst.loc[ytl]['t'] for ytl in range_num_of_signals]
ytick_labels=['']
ytick_labels.extend(ytls)

# comment the line below, if you want only signal numbers displayed on y-axis instead of timestamps
ax.set_yticklabels(ytick_labels)


ax.tick_params(labelsize=8)
ax.set_xlabel('Frequency', fontsize=9)
ax.set_zlabel('VelocityFFT_Amplitude', fontsize=9)
plt.yticks(rotation=-90, fontsize=6)

fig.subplots_adjust(bottom=0.3)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Zoomable plot

In [34]:
# provide signal numbers here
signals_to_plot = [95, 96, 97, 98, 99, 100, 101, 102]

plot_mapping(xys, signals_to_plot)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …